# Lab1 — PyTorch Foundations for Computer Vision

**Course**: Deep Learning for Image Analysis

**Class**: M2 IASD App  

**Professor**: Mehyar MLAWEH

---

## Objectives
By the end of this lab, you should be able to:

- Understand how **neurons and layers** are implemented in PyTorch
- Manipulate **tensors** and reason about shapes
- Use **autograd** to compute gradients
- Implement a **training loop** yourself
- Connect theory (neurons, loss, backprop) to actual code

⚠️ This notebook is **intentionally incomplete**.  
Whenever you see **`# TODO`**, you are expected to write code.


**Deadline:** 🗓️ **Saturday, February 7th (23:59)**

## 🤖 A small (honest) note before you start

Let’s be real for a second.

 I know you **can use LLMs (ChatGPT, Copilot, Claude, etc.)** to help you with this lab.  
And yes, **I use them too**, so don’t worry 😄

👉 **You are allowed to use AI tools.**  
But here’s the deal:

- Don’t just **copy–paste** code you don’t understand  
- Take time to **read, question, and modify** what the model gives you  
- If you can solve a block **by yourself, without AI**, that’s excellent

Remember:

> AI can write code for you, but **only you can understand it** — and understanding is what matters for exams, projects, and real work.

Use these tools **as assistants, not as replacements for thinking**.

---

## 📚 Useful documentation (highly recommended)

You will often find answers faster (and more reliably) by checking the official documentation:

- **PyTorch main documentation**  
  https://pytorch.org/docs/stable/index.html

- **PyTorch tensors**  
  https://pytorch.org/docs/stable/tensors.html

- **Neural network modules (`torch.nn`)**  
  https://pytorch.org/docs/stable/nn.html

- **Loss functions** (`BCEWithLogitsLoss`, CrossEntropy, etc.)  
  https://pytorch.org/docs/stable/nn.html#loss-functions

- **Optimizers** (`SGD`, `Adam`, …)  
  https://pytorch.org/docs/stable/optim.html

If you learn how to **navigate the documentation**, you are already thinking like a real AI engineer 👌

---

## PART I

## 0) Colab setup — GPU check

**Instructions**
1. In Colab: `Runtime → Change runtime type to GPU T4`
2. Select **GPU**
3. Save and restart runtime

Then run the cell below.


In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
device = "cuda"
print("Using device:", device)


PyTorch version: 2.9.0+cu126
CUDA available: True
Using device: cuda


## 1) Imports and reproducibility


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(7)


## 2) PyTorch tensors and shapes

Tensors are multi-dimensional arrays that support:
- GPU acceleration
- automatic differentiation

Understanding **shapes** is critical in deep learning.


In [ ]:
# Examples
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.randn(4, 5)

print("a shape:", a.shape)
print("b shape:", b.shape)


a shape: torch.Size([3])
b shape: torch.Size([4, 5])


### 🔍 Question (answer inside the markdown)
- How many dimensions does tensor `b` have?
- What does each dimension represent conceptually?

Tensor b has 2 dimensions.
The first dimension (4) represents rows/samples, and the second (5) represents columns/features.

### ✅Tensor operations

Complete the following:

1. Create a tensor `x` of shape `(8, 3)` with random values  
2. Compute:
   - the **mean of each column**
   - the **L2 norm of each row**
3. Normalize `x` **row-wise** using the L2 norm

In [ ]:
import torch

# TODO: create x
x = torch.randn(8, 3)

# TODO: column mean
col_mean = x.mean(dim=0)

# TODO: row-wise L2 norm
row_norm = torch.norm(x, p=2, dim=1, keepdim=True)

# TODO: normalized tensor
x_normalized = x / row_norm

print(x.shape, col_mean.shape, row_norm.shape, x_normalized.shape)


torch.Size([8, 3]) torch.Size([3]) torch.Size([8, 1]) torch.Size([8, 3])


## 3) Artificial neuron — from math to code

A neuron computes:

$$
z = \sum_i w_i x_i + b
$$

Then applies an activation function:

$$
y = g(z)
$$

This section connects directly to the theory seen in class.


In [ ]:
x = torch.tensor([1.0, -2.0, 3.0])
w = torch.tensor([0.2, 0.4, -0.1])
b = torch.tensor(0.1)

z = torch.sum(x * w) + b
z


tensor(-0.8000)

### Activation functions

1. Implement **ReLU**
2. Implement **Sigmoid**
3. Apply both to `z` and compare the outputs

Which activation preserves negative values?
Neither ReLU nor Sigmoid preserves negative values.

In [ ]:
import torch

# TODO
def relu(z):
    return torch.maximum(z, torch.tensor(0.0))

def sigmoid(z):
    return 1 / (1 + torch.exp(-z))

# Example input
z = torch.randn(5)

y_relu = relu(z)
y_sigmoid = sigmoid(z)

y_relu, y_sigmoid


(tensor([0.0000, 0.0000, 0.0000, 1.0056, 0.0000]),
 tensor([0.4807, 0.2845, 0.1308, 0.7322, 0.3330]))

## 4) Autograd and gradients

PyTorch uses **automatic differentiation** to compute gradients
using the **chain rule** (backpropagation).


In [ ]:
x = torch.tensor([1.0, 2.0, -1.0], requires_grad=True)
w = torch.tensor([0.5, -0.3, 0.8], requires_grad=True)
b = torch.tensor(0.2, requires_grad=True)

z = torch.sum(x * w) + b
loss = (z - 1.0) ** 2

loss.backward()

print("loss:", loss.item())
print("grad w:", w.grad)
print("grad b:", b.grad)


loss: 2.890000104904175
grad w: tensor([-3.4000, -6.8000,  3.4000])
grad b: tensor(-3.4000)


### 🔍 Conceptual question

- If `b.grad > 0`, should `b` increase or decrease after a gradient descent step?
Explain **why** in one sentence.

If b.grad > 0, b should decrease, because gradient descent subtracts the gradient to minimize the loss

## 5) Toy classification dataset

We create a **linearly separable** dataset.

Label rule:
- class = 1 if `x₁ + x₂ + x₃ > 0`
- else class = 0

This mimics a very simple classification problem.


In [ ]:
# TODO: generate a dataset
N = 500

# Features: random values
X = torch.randn(N, 3)

# Labels using the rule
y = (X.sum(dim=1) > 0).float().unsqueeze(1)  # shape (N, 1)

# TODO: split into train / validation
split = int(0.8 * N)

X_train = X[:split]
X_val = X[split:]

y_train = y[:split]
y_val = y[split:]


## 6) Model definition

We define a small **MLP** (fully-connected network):

`3 → 16 → 8 → 1`

Activation: ReLU  
Output: raw logits (no sigmoid)


In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 16),   # 3 → 16
            nn.ReLU(),         # ReLU
            nn.Linear(16, 8),   # 16 → 8
            nn.ReLU(),         # ReLU
            nn.Linear(8, 1)     # 8 → 1 (logits)
        )

    def forward(self, x):
        return self.net(x)

# TODO: create model and move it to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP().to(device)


###  parameters

1. Compute **by hand** the total number of parameters
2. Verify your answer using PyTorch


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
total_params


209

## 7) Training loop

You must complete the full training loop:
- forward pass
- loss computation
- backward pass
- optimizer step

Loss: `BCEWithLogitsLoss`
Optimizer: `SGD`


In [ ]:
# TODO: move data to device
X_train_d = X_train.to(device)
y_train_d = y_train.to(device)
X_val_d = X_val.to(device)
y_val_d = y_val.to(device)

# TODO: define loss and optimizer

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(50):
    model.train()
    optimizer.zero_grad()

    # TODO: forward
    logits = model(X_train_d)

    # TODO: loss
    loss = criterion(logits, y_train_d)

    # TODO: backward
    loss.backward()

    # TODO: update
    optimizer.step()

    if epoch % 5 == 0:
        print("Epoch", epoch, "| loss =", float(loss))


Epoch 0 | loss = 0.05529891699552536
Epoch 5 | loss = 0.05441880226135254
Epoch 10 | loss = 0.05357513204216957
Epoch 15 | loss = 0.05276401340961456
Epoch 20 | loss = 0.05198308825492859
Epoch 25 | loss = 0.05123031884431839
Epoch 30 | loss = 0.050504233688116074
Epoch 35 | loss = 0.049803171306848526
Epoch 40 | loss = 0.0491260327398777
Epoch 45 | loss = 0.04847106710076332


## 8) Evaluation

1. Apply `sigmoid` to the logits
2. Convert probabilities to predictions
3. Compute **accuracy** on the validation set


In [ ]:
#TODO: evaluation
with torch.no_grad():
       logits = model(X_val_d)
       probs = torch.sigmoid(logits)
       preds = (probs > 0.5).float()
       accuracy = (preds == y_val_d).float().mean()

accuracy = accuracy.item()
accuracy


1.0

## 9) Reflection questions (answer inside the markdown)

1. Why do we **not** apply sigmoid inside the model?

We typically don't apply sigmoid directly in the model's output layer when using BCEWithLogitsLoss because this loss function combines the sigmoid activation and the binary cross-entropy loss into a single, numerically stable operation. This prevents issues like vanishing gradients that can occur when computing sigmoid and then loss separately.

2. What would happen if we removed all ReLU activations?

If all ReLU activations were removed, the network would become a simple linear model, regardless of the number of layers. Without non-linear activation functions, the composition of linear transformations is still a linear transformation, limiting the model's ability to learn complex, non-linear relationships in the data.

3. How does this toy problem relate to image classification?

This toy problem provides a simplified, foundational understanding of the core concepts used in image classification, such as defining a neural network architecture (MLP), handling data, calculating loss, and performing optimization (training loop). While image classification involves more complex inputs (pixels), specialized layers (CNNs), and larger datasets, the underlying principles of forward pass, backpropagation, and gradient-based learning remain the same.

Write short answers (2–3 lines each).


## 10) Bridge to Computer Vision

So far:
- inputs = vectors of size 3
- layers = fully-connected

Next session:
- inputs = images `(B, C, H, W)`
- layers = convolutions
- same training logic

👉 **Architecture changes, learning principles stay the same.**


## Part II — Training on MNIST

Check the next notebook